In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go

import geopandas
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
import seaborn as sb
import plotly
import dash
import plotly.offline as ply
from plotly.graph_objs import *
from scipy.io import netcdf
import scipy.stats as stat

origins = ["DR_Congo", "Afghanistan", "Syria", "Myanmar", "Sudan"]
destinations = ["USA", "UK", "France", "Canada", "Italy", "Germany"]

org_codes = ["COD", "AFG", "SYR", "MMR", "SDN"]
dest_codes = ["USA", "GBR", "FRA", "CAN", "ITA","DEU"]

years = [2000 + i for i in range(19)]

features = ['destination',
 'Origin',
 'applied',
 'accepted',
 'Rejected',
 'decisions',
 'Year',
 'Month',
 'Value',
 'deaths',
 'last_month',
 'two_months_ago',
 'next_month',
 'two_months_later']


months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sept", "Oct", "Nov", "Dec"]

combos = []
for year in years:
    for month in months:
        combos.append((month, year))


coordinates = {}
coordinates["Syria"] = (34.8, 38.997)
coordinates["DR_Congo"] = (-4.0383, 21.759)
coordinates["Afghanistan"] = (33.939, 67.71)
coordinates["Sudan"] = (12.863, 30.218)
coordinates["Myanmar"] = (21.9162, 95.956)

coordinates["USA"] = (37.09, -95.713)
coordinates["UK"] = (55.378, -3.436)
coordinates["France"] = (46.228, 2.214)
coordinates["Germany"] = (51.166, 10.4515)
coordinates["Italy"] = (41.872, 12.5674)
coordinates["Canada"] = (56.1304, -106.3468)

pairs = []
for origin in origins:
    for destination in destinations:
        pairs.append((origin, destination))

# Read in data
df = pd.read_csv('./data/all_merged.csv', skipinitialspace=True)

symbols = {'DR_Congo': 'triangle-down', 'Sudan': 'square', 'Syria': 'cross',\
            'Myanmar':'star', 'Afghanistan': 'circle'}

In [2]:
for month in months:
    df.ix[df.Month.str.contains(month), ['Month']] = month
month_ids = []
markers = []
origin_ids = []
for index, row in df.iterrows():
    markers.append(symbols[row["Origin"]])
    month = row.Month
    month_ids.append(months.index(month) + 1)
    origin_ids.append(origins.index(row.Origin))
df['shapes'] = markers
df['month_id'] = month_ids
df['month_number'] =12*(df['Year']-2000) + df['month_id']
df['origin_id'] = origin_ids

/Users/jasonterry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated



In [3]:
grouped = df.groupby('destination')

In [4]:
group_dict = {}
for (name, df) in grouped:
    group_dict[name] = df

In [5]:
group_dict[list(group_dict.keys())[0]].sample(5)

,Unnamed: 0,destination,Origin,applied,accepted,Rejected,decisions,Year,Month,Value,deaths,shapes,month_id,month_number,origin_id
6808,6808,Canada,Myanmar,1.000000e-16,1.000000e-16,1.000000e-16,8.880891e-17,2017,Mar,6.504205e-17,500.0,star,3,207,3
1762,1762,Canada,Myanmar,3.100000e+01,2.200000e+01,1.000000e+00,2.600000e+01,2004,Apr,5.000000e+00,27.0,star,4,52,3
4806,4806,Canada,Syria,3.500000e+02,1.550000e+02,3.400000e+01,1.960000e+02,2012,Sept,3.600000e+01,38505.0,cross,9,153,2
3142,3142,Canada,Afghanistan,3.080000e+02,1.210000e+02,2.900000e+01,1.760000e+02,2007,Jun,1.400000e+01,7017.0,circle,6,90,1
5421,5421,Canada,DR_Congo,2.790000e+02,2.110000e+02,1.730000e+02,4.280000e+02,2013,Aug,3.200000e+01,1573.0,triangle-down,8,164,0


## Plot with dropdown menu that chooses origin. Shows all refugees for each country

In [16]:
fig = go.FigureWidget(
    data=[dict(
    
        x = df.query('Origin == @origin')['month_number'],
        y = df.query('Origin == @origin')['Value'],
        name = origin, 
        mode = 'markers'
    ) for origin in origins
    ]
)
years = [2000 + i for i in range(18)]
marker_places= [12*i for i in range(18)]
tick_labs = [str(year) for year in years]
# scatter.marker.color = df['Origin']
fig.layout.hovermode = 'closest'
fig.layout.yaxis.title = 'Refugees Taken In'
fig.layout.xaxis.title = 'Month'
fig.layout.xaxis.ticktext = tick_labs
fig.layout.xaxis.tickvals = marker_places
# scatter = fig.data[0]
# scatter.marker.size = 3
# scatter.marker.opacity = 0.2

In [10]:
fig.data[1]['name']

'Afghanistan'

In [17]:
from ipywidgets import interact

@interact(menu = destinations)
def update_arguments(menu=['Canada']):
    for i in range(len(fig.data)):
        origin = fig.data[i]['name']
        fig.data[i].x = group_dict[menu].query('Origin == @origin')\
                        .month_number
        fig.data[i].y = group_dict[menu].query('Origin == @origin')\
                        .Value + \
                        np.random.rand(len(group_dict[menu]\
                                           .query('Origin == @origin')\
                        .month_number))
    fig.layout.title = 'Refugees taken in by ' + menu
fig

interactive(children=(Dropdown(description='menu', options=('USA', 'UK', 'France', 'Canada', 'Italy', 'Germany…

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 'DR_Congo',
              'type': 'scatt…